In [1]:
from calc_cover_time import *
def get_all_drones_missile_cover_times_sum(drone_positions, drone_params, missile_trajs, true_target_corners, smoke_radius, t_list, debug=False):
    """
    drone_positions: dict, 5个无人机位置
    drone_params: shape (5, 8)，每个无人机的参数 [speed, angle, throw1, throw2, throw3, burst1, burst2, burst3]
    missile_trajs: list of 3 np.ndarray，每个为(3, len(t_list))，三个导弹轨迹
    true_target_corners: list of 4 np.array([x, y, z])
    smoke_radius: float
    t_list: np.array
    debug: bool，是否输出详细遮蔽信息

    返回: 所有导弹被所有无人机遮蔽时间的总和（float）
    """
    num_drones = 5
    num_missiles = len(missile_trajs)
    covered_by_drone = np.zeros((num_drones, num_missiles, len(t_list)), dtype=bool)

    for i in range(num_drones):
        speed = drone_params[i, 0]
        angle = drone_params[i, 1]
        throw_times = np.sort(drone_params[i, 2:5])
        burst_delays = drone_params[i, 5:8]
        smoke_centers = get_smoke_center_multi(
            drone_positions[i], angle, speed, throw_times, burst_delays, t_list
        )
        covered_bool = get_missile_cover_bool_multi_corners(
            smoke_centers, missile_trajs, true_target_corners, smoke_radius, t_list, debug=False
        )  # shape (3, len(t_list))
        covered_by_drone[i] = covered_bool

    # 合并所有无人机的遮蔽情况
    covered_all = np.any(covered_by_drone, axis=0)  # shape (3, len(t_list))
    cover_times = np.sum(covered_all, axis=1) * (t_list[1] - t_list[0])  # shape (3,)
    total_cover_time = np.sum(cover_times)

    if debug:
        for m in range(num_missiles):
            print(f"\n导弹{m+1}被遮蔽区间（按无人机分）：")
            for i in range(num_drones):
                intervals = get_cover_intervals(covered_by_drone[i, m], t_list)
                if intervals:
                    print(f"  无人机{i+1}:")
                    for start, end in intervals:
                        print(f"    {start:.6f}s ~ {end:.6f}s")
            print(f"导弹{m+1}总遮蔽时间: {cover_times[m]:.6f}s")
        print(f"\n所有导弹被所有无人机遮蔽时间总和: {total_cover_time:.6f}s")
    return total_cover_time

In [2]:
params = np.array([
    1.39947949e+02, 3.13366040e+00, 6.56731053e-03, 3.64855823e+00, 1.54525393e+01, 3.60924375e+00, 5.31779428e+00, 7.98774517e+00,
    114.62860037,   4.98699612,   6.01566818,   7.21914132,   8.69434875,   2.78934729,   5.27433416,   0.39108412,
    102.20517464,   2.20408204,  28.25700456,  29.62434123,  31.26438227,   7.66011547,   8.07963526,   7.67639008,
    125.33515088,   4.64755589,   3.60006083,  28.75823868,  30.44272478,  11.98307911,  18.06998011,   1.181712,
    139.51705493,  2.13999875,    11.40811482,  12.61961772, 17.82003334, 2.36261189, 4.48262589, 2.396845741
])


In [ ]:
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import dual_annealing
def optimize_drones_params(params, drone_positions, missile_trajs, true_target_corners, smoke_radius, t_list):
    """
    params: shape (40,)，初始参数
    drone_positions: dict, 5个无人机位置
    missile_trajs: list of 3 np.ndarray，每个为(3, len(t_list))
    true_target_corners: list of 4 np.array([x, y, z])
    smoke_radius: float
    t_list: np.array
    返回: 优化后的参数、最大遮蔽总时间
    """
    # 参数reshape为(5,8)
    params = np.array(params)
    bounds = []
    for i, p in enumerate(params):
        lower = p * 0.9
        upper = p * 1.1
        bounds.append((lower, upper))

    def objective(x):
        try:
            x = np.array(x).reshape(5, 8)
            return -get_all_drones_missile_cover_times_sum(
                drone_positions, x, missile_trajs, true_target_corners, smoke_radius, t_list, debug=False
            )
        except AssertionError:
            return 1e6  # 返回极差值，惩罚不合法参数
    result = dual_annealing(
        func=objective,
        bounds=bounds,
        maxiter=100,
        initial_temp=1000,
    )

    best_x = result.x.reshape(5, 8)
    print("优化结束，最优参数如下：")
    print(best_x)
    print("\n详细遮蔽信息：")
    total_cover_time = get_all_drones_missile_cover_times_sum(
        drone_positions, best_x, missile_trajs, true_target_corners, smoke_radius, t_list, debug=True
    )
    print(f"\n最大遮蔽总时间: {total_cover_time:.6f}s")
    return best_x, total_cover_time

# 用法示例
best_x, total_cover_time = optimize_drones_params(params, drone_pos, missile_trajs, true_target_corners, smoke_R, t_list)

In [ ]:
best_x = np.array(params).reshape(5, 8)
print("优化结束，最优参数如下：")
print(best_x)
print("\n详细遮蔽信息：")
total_cover_time = get_all_drones_missile_cover_times_sum(
    drone_pos, best_x, missile_trajs, true_target_corners, 10, t_list, debug=True
)
print(f"\n最大遮蔽总时间: {total_cover_time:.6f}s")

优化结束，最优参数如下：
[[1.39947949e+02 3.13366040e+00 6.56731053e-03 3.64855823e+00
  1.54525393e+01 3.60924375e+00 5.31779428e+00 7.98774517e+00]
 [1.14628600e+02 4.98699612e+00 6.01566818e+00 7.21914132e+00
  8.69434875e+00 2.78934729e+00 5.27433416e+00 3.91084120e-01]
 [1.02205175e+02 2.20408204e+00 2.82570046e+01 2.96243412e+01
  3.12643823e+01 7.66011547e+00 8.07963526e+00 7.67639008e+00]
 [1.25335151e+02 4.64755589e+00 3.60006083e+00 2.87582387e+01
  3.04427248e+01 1.19830791e+01 1.80699801e+01 1.18171200e+00]
 [1.39517055e+02 2.13999875e+00 1.14081148e+01 1.26196177e+01
  1.78200333e+01 2.36261189e+00 4.48262589e+00 2.39684574e+00]]

详细遮蔽信息：

导弹1被遮蔽区间（按无人机分）：
  无人机1:
    4.97s ~ 11.57s
  无人机2:
    18.64s ~ 22.42s
  无人机3:
    38.88s ~ 41.19s
  无人机4:
    16.22s ~ 19.95s
  无人机5:
    24.48s ~ 27.75s
导弹1总遮蔽时间: 18.39s

导弹2被遮蔽区间（按无人机分）：
  无人机2:
    9.37s ~ 12.95s
    19.81s ~ 22.54s
  无人机3:
    39.79s ~ 41.98s
  无人机5:
    20.40s ~ 23.74s
导弹2总遮蔽时间: 9.70s

导弹3被遮蔽区间（按无人机分）：
  无人机3:
    36.69s ~ 38